In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("E:/baselines/SP500_new.csv")
#data = data.drop(columns=['Adj Close'])
#data.rename(columns={'Date':'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume', 'company':'stockcode'}, inplace=True)
data.rename(columns={'dt':'date', 'kdcode':'stockcode'}, inplace=True)

In [3]:
data['date'] = pd.to_datetime(data['date'])
grouped_df = data.groupby('stockcode')
l = data['date'].nunique()
def filter_groups(group):
    return len(group) == l
df = grouped_df.filter(filter_groups)
df

,stockcode,date,open,high,low,close,volume
0,A,2018-01-02,67.419998,67.889999,67.339996,67.599998,1047800.0
1,A,2018-01-03,67.620003,69.489998,67.599998,69.320000,1698900.0
2,A,2018-01-04,69.540001,69.820000,68.779999,68.800003,2230700.0
3,A,2018-01-05,68.730003,70.099998,68.730003,69.900002,1632500.0
4,A,2018-01-08,69.730003,70.330002,69.550003,70.050003,1613400.0
...,...,...,...,...,...,...,...
734391,ZTS,2023-12-21,196.000000,196.979996,193.899994,194.660004,2118600.0
734392,ZTS,2023-12-22,195.320007,195.910004,192.740005,194.979996,1548400.0
734393,ZTS,2023-12-26,194.880005,196.339996,194.089996,195.500000,814600.0
734394,ZTS,2023-12-27,195.410004,197.009995,194.740005,196.899994,766400.0


In [4]:
df['stockcode'].nunique()

487

In [5]:
df['ret_1'] = df['close'] / df['close'].shift(1)
df['ret_1'] = df['ret_1'].fillna(1)
df['label'] = df['ret_1'].shift(-1)
df['label'] = df['label'].fillna(1)
df

,stockcode,date,open,high,low,close,volume,ret_1,label
0,A,2018-01-02,67.419998,67.889999,67.339996,67.599998,1047800.0,1.000000,1.025444
1,A,2018-01-03,67.620003,69.489998,67.599998,69.320000,1698900.0,1.025444,0.992499
2,A,2018-01-04,69.540001,69.820000,68.779999,68.800003,2230700.0,0.992499,1.015988
3,A,2018-01-05,68.730003,70.099998,68.730003,69.900002,1632500.0,1.015988,1.002146
4,A,2018-01-08,69.730003,70.330002,69.550003,70.050003,1613400.0,1.002146,1.024554
...,...,...,...,...,...,...,...,...,...
734391,ZTS,2023-12-21,196.000000,196.979996,193.899994,194.660004,2118600.0,1.000154,1.001644
734392,ZTS,2023-12-22,195.320007,195.910004,192.740005,194.979996,1548400.0,1.001644,1.002667
734393,ZTS,2023-12-26,194.880005,196.339996,194.089996,195.500000,814600.0,1.002667,1.007161
734394,ZTS,2023-12-27,195.410004,197.009995,194.740005,196.899994,766400.0,1.007161,1.001321


In [6]:
df.loc[df['date'] == '2018-01-02', 'ret_1'] = 1
df.loc[df['date'] == '2023-12-28', 'label'] = 1

In [7]:
df

,stockcode,date,open,high,low,close,volume,ret_1,label
0,A,2018-01-02,67.419998,67.889999,67.339996,67.599998,1047800.0,1.000000,1.025444
1,A,2018-01-03,67.620003,69.489998,67.599998,69.320000,1698900.0,1.025444,0.992499
2,A,2018-01-04,69.540001,69.820000,68.779999,68.800003,2230700.0,0.992499,1.015988
3,A,2018-01-05,68.730003,70.099998,68.730003,69.900002,1632500.0,1.015988,1.002146
4,A,2018-01-08,69.730003,70.330002,69.550003,70.050003,1613400.0,1.002146,1.024554
...,...,...,...,...,...,...,...,...,...
734391,ZTS,2023-12-21,196.000000,196.979996,193.899994,194.660004,2118600.0,1.000154,1.001644
734392,ZTS,2023-12-22,195.320007,195.910004,192.740005,194.979996,1548400.0,1.001644,1.002667
734393,ZTS,2023-12-26,194.880005,196.339996,194.089996,195.500000,814600.0,1.002667,1.007161
734394,ZTS,2023-12-27,195.410004,197.009995,194.740005,196.899994,766400.0,1.007161,1.001321


In [10]:
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols_to_normalize = ['close', 'open', 'high', 'low', 'volume']
# unique_stockcodes = df['stockcode'].unique()[:100]

# df = df[df['stockcode'].isin(unique_stockcodes)]

for stock in tqdm(df['stockcode'].unique()):
    tmp = df[df['stockcode'] == stock].reset_index(drop=True)
    tmp[cols_to_normalize] = scaler.fit_transform(tmp[cols_to_normalize])
    tmp.to_pickle(f"sp500/{stock}.pkl")

100%|██████████| 487/487 [00:19<00:00, 25.45it/s]


In [11]:
df.to_csv('sp500_fea.csv', index=None)

In [ ]:
df['stockcode'].unique()

In [ ]:
pom = pd.read_pickle("zz500/000009.SZ.pkl")
pom